## EE-361M Introduction to Data Mining
## Assignment #5
## Due: Thursday, Apr 14, 2016 by 2pm; Total points: 60


Your homework should be written in a **Jupyter notebook** (if this isn't possible, let me know). Please use this naming format for your notebook you submit: **Group(Group Num)_HW(HW Number).ipynb**. For example, Group1_HW1.ipynb. Homeworks should be submitted through Canvas in your **groups of 3 from the first homework**. If groups need to be adjusted please contact the TA. Also, please make sure your code runs and the graphics (and anything else) are displayed in your notebook before submitting.

In [60]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pydataset import data
from sklearn import tree, svm, grid_search
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

%matplotlib inline

# Question 1 (1+1+5+3+3+2=15pts) - Logistic Regression

In this question we will be predicting whether someone will have an affair! Yes - there is data on this. See below on how to import the data.
1. Convert naffairs to a binary variable hadAffair which is 1 if had an affair and zero otherwise
2. Split the data into training and test. Use 42 as random seed and use 1/3rd of the data for testing. Our y variable is hadAffair and X matrix includes all the other variables except naffairs.
3. Train a logistic regression with almost no regularization (pass l2 (ridge) to penalty and 1,000 to the C parameter which is the inverse of regularization strength lambda. This essentially does l2 regularization but applies very little weight to the penalty term) and report the confusion matrix on the test data. Also report the accuracy for the "no affairs" class, the affairs class, and the average per-class accuracy on the test data. Average per-class accuracy is described in this [post](http://blog.dato.com/how-to-evaluate-machine-learning-models-part-2a-classification-metrics).
4. Repeat step 3 except use l2 penalty with Cs of [.001, .01,0.1, 1]. You will want to use k-fold cross validation to select the best parameter. To evaluate which parameter is best, maximize the average per-class accuracy. To help with this task, check out [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and how to make your own [custom scorer](http://scikit-learn.org/stable/modules/model_evaluation.html).
5. Repeat question 4 except use l1 (i.e. Lasso) instead of l2 as the penalty type.
6. Which model produces the best average per-class accuracy? Why do you think this is the case? How do the models handle the different classes, and why is this so?

In [4]:
from pydataset import data
df = data('affairs')

# Question 2 (2+3+2+3=10pts) - Support Vector Classifier

This question will continue to use the data from question 1 - including the training and test split data.
1. Fit a support vector classifier using the standard options on [sklearn's SVC](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC). Report the confusion matrix on the test data. Also report the accuracy for the no affairs class, the affairs class, and the average per-class accuracy (same as question 1).
2. Repeat question 1 except use grid search to select the best value of C within this set: [0.001, 0.01, 0.1, 1,5,10,100] and try both a radial and polynomial kernel (thus trying 14 combinations). Choose the combination that maximizes the average per-class accuracy. Use 5 folds. Report the best model, the confusion matrix, the accuracy for the no affairs class, the affairs class, and the average per-class accuracy.
3. Briefly discuss the effect of different  C,  kernel combinations.
4. Discuss your results from parts 1 and 2 and mention how they differ from Question 1's results.

# Question 3 (2+1+3+1+3=10pts) - Regression Trees

This question is very similar to homework 4 question 1. Except now we will be using regression trees and not classification trees, and you will be addressing a regression problem (i.e., the independent variable "price" will not be binarized).

For this question, we will be using the housing dataset (see code below). 

1. Convert driveway, recroom, fullbase, gashw, airco, and prefarea to numeric dummy variables (1 for yes, zero for no)
2. Split the data into training and testing with a random seed of 42 and keeping 1/3rd of the data for testing
2. Fit a [decision tree regressor](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html#sklearn.tree.DecisionTreeRegressor) to predict price using all the data (your dummy variables plus bedrooms and bathrooms).
5. Report the root MSE on the test data.
6. How does the tree decide on a splitting point?

In [81]:
# part 1
df = data('Housing')
df_dummies = pd.get_dummies(df)
#df_dummies.head()
features = ['driveway_yes','recroom_yes', 'fullbase_yes','gashw_yes', 'airco_yes','prefarea_yes','bedrooms','bathrms']

# part 2
X = df_dummies[features].values
y = df_dummies['price'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

#part 3
clf = DecisionTreeRegressor()
result = clf.fit(X_train, y_train)
y_pred = result.predict(X_test)
#for i in range(0,10):
#    print "y test %.2f \t | diff: " % y_test[i], y_test[i]-y_pred[i]

#part 4
print "RegTree RMSE: %.2f\n" % np.sqrt(mean_squared_error(y_test, y_pred))

#part 5
print """By selecting binary splits that minimize the sum of the squared deviations from the mean in the two 
separate partitions, i.e. in each division, all datapoints in that division correspond to a certain value"""

RegTree RMSE: 21504.49

By selecting binary splits that minimize the sum of the squared deviations from the mean in the two 
separate partitions, i.e. in each division, all datapoints in that division correspond to a certain value


# Question 4 (2+5+3=10pts) - Support Vector Regression

This question will continue to use the data from question 4 - including the training and test split data.

1. Fit a support vector regression using the standard options on [sklearn's SVR](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html). Report the root MSE.
2. Repeat question 1 except use grid search to select the best value of C within this set: [0.001, 0.01, 0.1, 1,5,10,100] and try both a radial and polynomial kernel (thus trying 14 combinations). Choose the combination that minimizes MSE. Use 5 folds. Report the best model and the test root MSE.
4. Discuss your results from parts 1 and 2 and how they differ from Question 4 results.

In [85]:
#part 1
svm_reg_clf = svm.SVR()
result = svm_reg_clf.fit(X_train, y_train) 
y_pred = result.predict(X_test)
#print svm_reg_clf.score(X_test,y_test)
print "SVM Reg RMSE: %.2f" % np.sqrt(mean_squared_error(y_test, y_pred))

#part 2
parameters = {'kernel':('poly', 'rbf'), 'C': [0.001, 0.01, 0.1, 1,5,10,100],}
grid_clf = grid_search.GridSearchCV(svm_reg_clf, parameters,cv=5)
result = grid_clf.fit(X_train, y_train)
y_pred = result.predict(X_test)
print "GridReg RMSE: %.2f" % np.sqrt(mean_squared_error(y_test, y_pred))
print "Best Model:"
print grid_clf.best_params_

#part 3
print """Both the SVMReg and GridSearch CV RMSE scores were worse than a (unpruned!!!!) regression tree. RegTree (at least 
the last iteration I ran) produced an RMSE score of 21504.49, whereas SVMReg and GridReg produced RMSE vals of 
29005.43 and 22200.72, respectively. The lesson here is that more complex models may not always do better. """

SVM Reg RMSE: 29005.43
GridReg RMSE: 22200.72
Best Model:
{'kernel': 'poly', 'C': 100}
Both the SVMReg and GridSearch CV RMSE scores were worse than a (unpruned!!!!) regression tree. RegTree (at least 
the last iteration I ran) produced an RMSE score of 21504.49, whereas SVMReg and GridReg produced RMSE vals of 
29005.43 and 22200.72, respectively. The lesson here is that more complex models may not always do better. 


# Question 5 (3+2+2+2+2+2+2=15pts) - Random Forest

This question will also continue to use the data from Question 1.
1. Fit a random forest model grid searching over the following values: {'n_estimators':[10, 100, 1000], 'max_features': ['auto', 'sqrt', 'log2']}. Choose the combination that maximizes the average per-class accuracy. Use 5 folds. Report the best model, the confusion matrix, the accuracy for the no affairs class, the affairs class, and the average per-class accuracy.
2. What do the n_estimators and max_features parameters do?
3. Report the features in order of importance based on the model used in part 1
4. Repeat question 1 using an AdaBoostClassifier and grid search over the following values: {'n_estimators':[50, 500, 5000], 'learning_rate': [.001, .01, .1]}
5. What does the learning_rate parameter do?
6. Report the features in order of importance based on the model used in part 4
7. Compare the results in part 1 and 4 and questions 2 and 3